In [2]:
import os
import requests
import zipfile
import xml.etree.ElementTree as ET
import re

In [3]:
import re

# Ange filnamn
LINES_FILE = "movie_lines.txt"
CONVERSATIONS_FILE = "movie_conversations.txt"
OUTPUT_FILE = "cleaned_movie_dialogues.txt"

def load_lines(file_path):
    """ Läser in repliker och skapar en ordbok {LID: text} """
    line_dict = {}
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                line_id, text = parts[0], parts[4]
                line_dict[line_id] = text
    return line_dict

def extract_conversations(lines_dict, file_path, output_file):
    """ Läser in `movie_conversations.txt`, ersätter LIDs med repliker och sparar ren text """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f, open(output_file, "w", encoding="utf-8") as out:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                line_ids = eval(parts[3])  # Konvertera sträng till lista ['L194', 'L195']
                conversation = " ".join(lines_dict.get(lid, "") for lid in line_ids)
                cleaned_text = re.sub(r"[^A-Za-z0-9.,!?;:\'\" ]+", " ", conversation)  # Rensa text
                out.write(cleaned_text.lower() + "\n")  # Konvertera till lowercase och spara

    print(f"✅ Sparat rensade dialoger i {output_file}")

# Kör skriptet
lines_dict = load_lines(LINES_FILE)
extract_conversations(lines_dict, CONVERSATIONS_FILE, OUTPUT_FILE)


✅ Sparat rensade dialoger i cleaned_movie_dialogues.txt
